In [18]:
%load_ext autoreload
%autoreload 2

import torch
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import of clean dataloader

In [32]:
from data_utils.data_loaders import ClassifierLoader
dl = ClassifierLoader(
    images_dir='../../data/processed/COCO-2017/CocoMultiClassifier/train/images/',
    batch_size=96,
    csv_path='../../data/processed/COCO-2017/CocoMultiClassifier/train/labels.csv',
    save_out_dir = "../../res/classifiers/classifier_multi/trainer/labels/",
    shuffle=True,
    validation_split=0.15,
    num_workers=6,
    balance_train=True,
    balance_methods=[
      'oversample',
      'make_0_half'  
    ],
    labels={
        "0": "none",
        "1": "person",
        "2": "bicycle",
        "3": "car",
        "4": "cat",
        "5": "dog"
    },
    balance_max_multiplicity=None
)
dl_train = dl.get_train_loader()

# Function to calculate mean and std of the dataset

In [36]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in tqdm(dataloader, desc="Calculating mean and std"):
        # print(data)
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [37]:
get_mean_and_std(dl_train)

Calculating mean and std: 100%|██████████| 5956/5956 [13:57<00:00,  7.11it/s]


(tensor([0.4119, 0.3770, 0.3452]), tensor([0.2832, 0.2707, 0.2711]))

# Check if works :) 

In [38]:
from data_utils.data_loaders import ClassifierLoader
dl = ClassifierLoader(
    images_dir='../../data/processed/COCO-2017/CocoMultiClassifier/train/images/',
    batch_size=96,
    csv_path='../../data/processed/COCO-2017/CocoMultiClassifier/train/labels.csv',
    save_out_dir = "../../res/classifiers/classifier_multi/trainer/labels/",
    shuffle=True,
    validation_split=0.15,
    transform_mean=[0.4119, 0.3770, 0.3452],
    transform_std=[0.2832, 0.2707, 0.2711],
    num_workers=6,
    balance_train=True,
    balance_methods=[
      'oversample',
      'make_0_half'  
    ],
    labels={
        "0": "none",
        "1": "person",
        "2": "bicycle",
        "3": "car",
        "4": "cat",
        "5": "dog"
    },
    balance_max_multiplicity=None
)
dl_train = dl.get_train_loader()

In [39]:
get_mean_and_std(dl_train)

Calculating mean and std: 100%|██████████| 5956/5956 [19:03<00:00,  5.21it/s]


(tensor([-5.2636e-05,  9.9283e-05, -1.4672e-04]),
 tensor([1.0000, 1.0000, 1.0002]))

# Results:
## Person:
+ "transform_mean": [0.5462,0.5803,0.6043]
+ "transform_std": [0.2952,0.2886,0.2965]
## Car:
+ "transform_mean": [0.5615,0.5820,0.5965]
+ "transform_std": [0.2911,0.2849,0.2908]
## Dog:
+ "transform_mean": [0.5243,0.5564,0.5869],
+ "transform_std": [0.2958,0.2923,0.3011]
## Bicycle:
+ "transform_mean": [0.5647,0.5906,0.6088]
+ "transform_std": [0.2894,0.283,0.2891]
## Cat:
+ "transform_mean": [0.5295,0.5672,0.5936]
+ "transform_std": [0.3045,0.3015,0.3107]
## Multi:
+ "transform_mean": [0.5438,0.5786,0.6030]
+ "transform_std": [0.2944,0.2882,0.2966]